In [ ]:
from google.colab import files
uploaded = files.upload()  # Manually upload the file

Saving ML_DEG_dataset.csv to ML_DEG_dataset.csv


In [ ]:
from google.colab import files
uploaded = files.upload()  # Manually upload the file

Saving GSE106582_gene_expression_renamed.csv to GSE106582_gene_expression_renamed.csv


In [ ]:
!pip install xgboost

# Feature Selection

# MI Feature Selection

## Grid Search CV

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
)
from xgboost import XGBClassifier
import warnings

warnings.filterwarnings("ignore")

# Load datasets
df_train = pd.read_csv("ML_DEG_dataset.csv")
columns_of_interest = ['ADH1B', 'CDH3', 'GUCA2B', 'PDE9A', 'ABCA8', 'GUCA2A', 'target']
df_train = df_train[columns_of_interest]
#df_train = df_train.iloc[:, 1:]
df_train = df_train[[col for col in df_train.columns if col != 'target'] + ['target']]
X_train = df_train.iloc[:, 0:-1]
y_train = df_train.iloc[:, -1]

df_test = pd.read_csv("GSE106582_gene_expression_renamed.csv")
df_test = df_test.iloc[:, 1:]
df_test = df_test[[col for col in df_test.columns if col != 'target'] + ['target']]
X_test = df_test.iloc[:, 0:-1]
y_test = df_test.iloc[:, -1]

# Ensure common features in both train and test sets
common_cols = set(X_train.columns).intersection(X_test.columns)
X_train_common = X_train[list(common_cols)]
X_test_common = X_test[list(common_cols)]

# Hyperparameter grid for each model
param_grids = {
    'Logistic Regression': {
        'C': [0.01, 0.1, 1, 10, 100],
        'penalty': ['l1', 'l2', 'elasticnet'],
        'solver': ['saga'],
        'max_iter': [500]
    },
    'K-Nearest Neighbors': {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    },
    'Random Forest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'Gradient Boosting': {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'min_samples_split': [2, 5]
    },
    'AdaBoost': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 1]
    },
    'Bagging': {
        'n_estimators': [50, 100, 200],
        'max_samples': [0.5, 0.75, 1.0],
        'max_features': [0.5, 0.75, 1.0],
        'bootstrap': [True, False]
    },
    'Extra Trees': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'XGBoost': {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
}

# Define models
models = {
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

# Dictionary to store best parameters
best_params = {}

# Tune and evaluate models
for name, model in models.items():
    print(f"Tuning {name}...")

    # Perform GridSearchCV
    grid_search = GridSearchCV(model, param_grids[name], cv=10, n_jobs=-1, scoring='accuracy')
    grid_search.fit(X_train_common, y_train)

    # Store best parameters
    best_params[name] = grid_search.best_params_
    print(f"Best parameters for {name}: {grid_search.best_params_}\n")

    # Train model with best parameters
    best_model = grid_search.best_estimator_
    best_model.fit(X_train_common, y_train)

    # Predict on test set
    predictions = best_model.predict(X_test_common)

    # Compute metrics
    test_accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted', zero_division=0)
    recall = recall_score(y_test, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=0)
    mcc = matthews_corrcoef(y_test, predictions)

    print(f"{name} Testing Accuracy: {test_accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"MCC: {mcc:.2f}")

    # Print confusion matrix and classification report
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("Classification Report:")
    print(classification_report(y_test, predictions))
    print("-" * 60)

# Print final best parameters
print("\nFinal Best Parameters for All Models:")
for model, params in best_params.items():
    print(f"{model}: {params}")

Tuning Logistic Regression...
Best parameters for Logistic Regression: {'C': 0.01, 'max_iter': 500, 'penalty': 'l2', 'solver': 'saga'}

Logistic Regression Testing Accuracy: 96.91%
Precision: 0.97
Recall: 0.97
F1-Score: 0.97
MCC: 0.94
Confusion Matrix:
[[114   3]
 [  3  74]]
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       117
           1       0.96      0.96      0.96        77

    accuracy                           0.97       194
   macro avg       0.97      0.97      0.97       194
weighted avg       0.97      0.97      0.97       194

------------------------------------------------------------
Tuning K-Nearest Neighbors...
Best parameters for K-Nearest Neighbors: {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'}

K-Nearest Neighbors Testing Accuracy: 95.36%
Precision: 0.95
Recall: 0.95
F1-Score: 0.95
MCC: 0.90
Confusion Matrix:
[[111   6]
 [  3  74]]
Classification Report:
             

# Random Search CV


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
)
from xgboost import XGBClassifier

import warnings

warnings.filterwarnings("ignore")

# Load datasets
df_train = pd.read_csv("ML_DEG_dataset.csv")
columns_of_interest = ['ADH1B', 'CDH3', 'GUCA2B', 'PDE9A', 'ABCA8', 'GUCA2A', 'target']
df_train = df_train[columns_of_interest]
#df_train = df_train.iloc[:, 1:]
df_train = df_train[[col for col in df_train.columns if col != 'target'] + ['target']]
X_train = df_train.iloc[:, 0:-1]
y_train = df_train.iloc[:, -1]

df_test = pd.read_csv("GSE106582_gene_expression_renamed.csv")
df_test = df_test.iloc[:, 1:]
df_test = df_test[[col for col in df_test.columns if col != 'target'] + ['target']]
X_test = df_test.iloc[:, 0:-1]
y_test = df_test.iloc[:, -1]

# Ensure common features in both train and test sets
common_cols = set(X_train.columns).intersection(X_test.columns)
X_train_common = X_train[list(common_cols)]
X_test_common = X_test[list(common_cols)]

# Hyperparameter grid for each model
param_grids = {
    'Logistic Regression': {
        'C': np.logspace(-3, 2, 10),
        'penalty': ['l1', 'l2', 'elasticnet'],
        'solver': ['saga'],
        'max_iter': [500, 1000, 5000],
        'l1_ratio': np.linspace(0, 1, 5).tolist()
    },
    'K-Nearest Neighbors': {
        'n_neighbors': range(3, 15),
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    },
    'Random Forest': {
        'n_estimators': range(100, 501, 100),
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'Gradient Boosting': {
        'n_estimators': range(100, 501, 100),
        'learning_rate': np.linspace(0.01, 0.2, 5),
        'max_depth': [3, 5, 7, 10],
        'subsample': [0.6, 0.8, 1.0],
        'min_samples_split': [2, 5, 10]
    },
    'AdaBoost': {
        'n_estimators': range(50, 301, 50),
        'learning_rate': np.linspace(0.01, 1, 5)
    },
    'Bagging': {
        'n_estimators': range(50, 301, 50),
        'max_samples': np.linspace(0.5, 1.0, 5),
        'max_features': np.linspace(0.5, 1.0, 5),
        'bootstrap': [True, False]
    },
    'Extra Trees': {
        'n_estimators': range(100, 501, 100),
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
     'XGBoost': {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
}

# Define models
models = {
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')

}

# Dictionary to store best parameters
best_params = {}

# Tune and evaluate models using RandomizedSearchCV
for name, model in models.items():
    print(f"Tuning {name}...")

    # Perform RandomizedSearchCV
    random_search = RandomizedSearchCV(model, param_grids[name],
                                       n_iter=20, cv=10,
                                       n_jobs=-1, scoring='accuracy', random_state=7)

    random_search.fit(X_train_common, y_train)

    # Store best parameters
    best_params[name] = random_search.best_params_
    print(f"Best parameters for {name}: {random_search.best_params_}\n")

    # Train model with best parameters
    best_model = random_search.best_estimator_
    best_model.fit(X_train_common, y_train)

    # Predict on test set
    predictions = best_model.predict(X_test_common)

    # Compute metrics
    test_accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted', zero_division=0)
    recall = recall_score(y_test, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=0)
    mcc = matthews_corrcoef(y_test, predictions)

    print(f"{name} Testing Accuracy: {test_accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"MCC: {mcc:.2f}")

    # Print confusion matrix and classification report
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("Classification Report:")
    print(classification_report(y_test, predictions))
    print("-" * 60)

# Print final best parameters
print("\nFinal Best Parameters for All Models:")
for model, params in best_params.items():
    print(f"{model}: {params}")

Tuning Logistic Regression...
Best parameters for Logistic Regression: {'solver': 'saga', 'penalty': 'l1', 'max_iter': 5000, 'l1_ratio': 0.5, 'C': np.float64(0.1668100537200059)}

Logistic Regression Testing Accuracy: 96.39%
Precision: 0.96
Recall: 0.96
F1-Score: 0.96
MCC: 0.92
Confusion Matrix:
[[114   3]
 [  4  73]]
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       117
           1       0.96      0.95      0.95        77

    accuracy                           0.96       194
   macro avg       0.96      0.96      0.96       194
weighted avg       0.96      0.96      0.96       194

------------------------------------------------------------
Tuning K-Nearest Neighbors...
Best parameters for K-Nearest Neighbors: {'weights': 'distance', 'n_neighbors': 4, 'metric': 'manhattan'}

K-Nearest Neighbors Testing Accuracy: 95.88%
Precision: 0.96
Recall: 0.96
F1-Score: 0.96
MCC: 0.91
Confusion Matrix:
[[112   5]
 [  

# Tuning for Hub genes

# Grid Search CV

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
)
from xgboost import XGBClassifier
import warnings

warnings.filterwarnings("ignore")

# Load datasets
df_train = pd.read_csv("ML_DEG_dataset.csv")
columns_of_interest = ['EHD2', 'TUBB', 'CSRP1', 'DPYSL3', 'TIMP2', 'RHOQ', 'ILK', 'SPARCL1', 'PALLD', 'PTRF',
                       'PPP3CB', 'MEF2C', 'RASL12', 'FLNA', 'PRNP', 'CAV2', 'SNCA', 'TNS1', 'TGFB1I1',
                       'SLIT2', 'ITPR1', 'LAMB2', 'RHOB', 'ACTN1','target']
df_train = df_train[columns_of_interest]
#df_train = df_train.iloc[:, 1:]
df_train = df_train[[col for col in df_train.columns if col != 'target'] + ['target']]
X_train = df_train.iloc[:, 0:-1]
y_train = df_train.iloc[:, -1]

df_test = pd.read_csv("GSE106582_gene_expression_renamed.csv")
df_test = df_test.iloc[:, 1:]
df_test = df_test[[col for col in df_test.columns if col != 'target'] + ['target']]
X_test = df_test.iloc[:, 0:-1]
y_test = df_test.iloc[:, -1]

# Ensure common features in both train and test sets
common_cols = set(X_train.columns).intersection(X_test.columns)
X_train_common = X_train[list(common_cols)]
X_test_common = X_test[list(common_cols)]

# Hyperparameter grid for each model
param_grids = {
    'Logistic Regression': {
        'C': [0.01, 0.1, 1, 10, 100],
        'penalty': ['l1', 'l2', 'elasticnet'],
        'solver': ['saga'],
        'max_iter': [500]
    },
    'K-Nearest Neighbors': {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    },
    'Random Forest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'Gradient Boosting': {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'min_samples_split': [2, 5]
    },
    'AdaBoost': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 1]
    },
    'Bagging': {
        'n_estimators': [50, 100, 200],
        'max_samples': [0.5, 0.75, 1.0],
        'max_features': [0.5, 0.75, 1.0],
        'bootstrap': [True, False]
    },
    'Extra Trees': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
     'XGBoost': {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
}

# Define models
models = {
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

# Dictionary to store best parameters
best_params = {}

# Tune and evaluate models
for name, model in models.items():
    print(f"Tuning {name}...")

    # Perform GridSearchCV
    grid_search = GridSearchCV(model, param_grids[name], cv=10, n_jobs=-1, scoring='accuracy')
    grid_search.fit(X_train_common, y_train)

    # Store best parameters
    best_params[name] = grid_search.best_params_
    print(f"Best parameters for {name}: {grid_search.best_params_}\n")

    # Train model with best parameters
    best_model = grid_search.best_estimator_
    best_model.fit(X_train_common, y_train)

    # Predict on test set
    predictions = best_model.predict(X_test_common)

    # Compute metrics
    test_accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted', zero_division=0)
    recall = recall_score(y_test, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=0)
    mcc = matthews_corrcoef(y_test, predictions)

    print(f"{name} Testing Accuracy: {test_accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"MCC: {mcc:.2f}")

    # Print confusion matrix and classification report
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("Classification Report:")
    print(classification_report(y_test, predictions))
    print("-" * 60)

# Print final best parameters
print("\nFinal Best Parameters for All Models:")
for model, params in best_params.items():
    print(f"{model}: {params}")

Tuning Logistic Regression...
Best parameters for Logistic Regression: {'C': 0.01, 'max_iter': 500, 'penalty': 'l2', 'solver': 'saga'}

Logistic Regression Testing Accuracy: 77.84%
Precision: 0.83
Recall: 0.78
F1-Score: 0.75
MCC: 0.56
Confusion Matrix:
[[116   1]
 [ 42  35]]
Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.99      0.84       117
           1       0.97      0.45      0.62        77

    accuracy                           0.78       194
   macro avg       0.85      0.72      0.73       194
weighted avg       0.83      0.78      0.75       194

------------------------------------------------------------
Tuning K-Nearest Neighbors...
Best parameters for K-Nearest Neighbors: {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'}

K-Nearest Neighbors Testing Accuracy: 48.97%
Precision: 0.70
Recall: 0.49
F1-Score: 0.42
MCC: 0.21
Confusion Matrix:
[[21 96]
 [ 3 74]]
Classification Report:
              pre

# Random Search CV

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
)
from xgboost import XGBClassifier
import warnings

warnings.filterwarnings("ignore")

# Load datasets
df_train = pd.read_csv("ML_DEG_dataset.csv")
columns_of_interest = ['EHD2', 'TUBB', 'CSRP1', 'DPYSL3', 'TIMP2', 'RHOQ', 'ILK', 'SPARCL1', 'PALLD', 'PTRF',
                       'PPP3CB', 'MEF2C', 'RASL12', 'FLNA', 'PRNP', 'CAV2', 'SNCA', 'TNS1', 'TGFB1I1',
                       'SLIT2', 'ITPR1', 'LAMB2', 'RHOB', 'ACTN1','target']
df_train = df_train[columns_of_interest]
#df_train = df_train.iloc[:, 1:]
df_train = df_train[[col for col in df_train.columns if col != 'target'] + ['target']]
X_train = df_train.iloc[:, 0:-1]
y_train = df_train.iloc[:, -1]

df_test = pd.read_csv("GSE106582_gene_expression_renamed.csv")
df_test = df_test.iloc[:, 1:]
df_test = df_test[[col for col in df_test.columns if col != 'target'] + ['target']]
X_test = df_test.iloc[:, 0:-1]
y_test = df_test.iloc[:, -1]

# Ensure common features in both train and test sets
common_cols = set(X_train.columns).intersection(X_test.columns)
X_train_common = X_train[list(common_cols)]
X_test_common = X_test[list(common_cols)]

# Hyperparameter grid for each model
param_grids = {
    'Logistic Regression': {
        'C': np.logspace(-3, 2, 10),
        'penalty': ['l1', 'l2', 'elasticnet'],
        'solver': ['saga'],
        'max_iter': [500, 1000, 5000],
        'l1_ratio': np.linspace(0, 1, 5).tolist()
    },
    'K-Nearest Neighbors': {
        'n_neighbors': range(3, 15),
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    },
    'Random Forest': {
        'n_estimators': range(100, 501, 100),
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'Gradient Boosting': {
        'n_estimators': range(100, 501, 100),
        'learning_rate': np.linspace(0.01, 0.2, 5),
        'max_depth': [3, 5, 7, 10],
        'subsample': [0.6, 0.8, 1.0],
        'min_samples_split': [2, 5, 10]
    },
    'AdaBoost': {
        'n_estimators': range(50, 301, 50),
        'learning_rate': np.linspace(0.01, 1, 5)
    },
    'Bagging': {
        'n_estimators': range(50, 301, 50),
        'max_samples': np.linspace(0.5, 1.0, 5),
        'max_features': np.linspace(0.5, 1.0, 5),
        'bootstrap': [True, False]
    },
    'Extra Trees': {
        'n_estimators': range(100, 501, 100),
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'XGBoost': {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
}

# Define models
models = {
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')

}

# Dictionary to store best parameters
best_params = {}

# Tune and evaluate models using RandomizedSearchCV
for name, model in models.items():
    print(f"Tuning {name}...")

    # Perform RandomizedSearchCV
    random_search = RandomizedSearchCV(model, param_grids[name],
                                       n_iter=20, cv=10,
                                       n_jobs=-1, scoring='accuracy', random_state=7)

    random_search.fit(X_train_common, y_train)

    # Store best parameters
    best_params[name] = random_search.best_params_
    print(f"Best parameters for {name}: {random_search.best_params_}\n")

    # Train model with best parameters
    best_model = random_search.best_estimator_
    best_model.fit(X_train_common, y_train)

    # Predict on test set
    predictions = best_model.predict(X_test_common)

    # Compute metrics
    test_accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted', zero_division=0)
    recall = recall_score(y_test, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=0)
    mcc = matthews_corrcoef(y_test, predictions)

    print(f"{name} Testing Accuracy: {test_accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"MCC: {mcc:.2f}")

    # Print confusion matrix and classification report
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("Classification Report:")
    print(classification_report(y_test, predictions))
    print("-" * 60)

# Print final best parameters
print("\nFinal Best Parameters for All Models:")
for model, params in best_params.items():
    print(f"{model}: {params}")

Tuning Logistic Regression...
Best parameters for Logistic Regression: {'solver': 'saga', 'penalty': 'l1', 'max_iter': 1000, 'l1_ratio': 0.75, 'C': np.float64(0.046415888336127795)}

Logistic Regression Testing Accuracy: 69.59%
Precision: 0.80
Recall: 0.70
F1-Score: 0.63
MCC: 0.39
Confusion Matrix:
[[117   0]
 [ 59  18]]
Classification Report:
              precision    recall  f1-score   support

           0       0.66      1.00      0.80       117
           1       1.00      0.23      0.38        77

    accuracy                           0.70       194
   macro avg       0.83      0.62      0.59       194
weighted avg       0.80      0.70      0.63       194

------------------------------------------------------------
Tuning K-Nearest Neighbors...
Best parameters for K-Nearest Neighbors: {'weights': 'distance', 'n_neighbors': 4, 'metric': 'manhattan'}

K-Nearest Neighbors Testing Accuracy: 49.48%
Precision: 0.69
Recall: 0.49
F1-Score: 0.43
MCC: 0.20
Confusion Matrix:
[[23 94]
 [ 

In [ ]:
['MT1M', 'ITM2A', 'VSNL1', 'BRCA2', 'AFF3', 'LGALS2', 'PCSK2', 'TMEM100', 'CDH3', 'LIFR', 'FAM107A', 'STMN2', 'ABCA8', 'CENPA', 'ADH1B','target']

In [ ]:
# Grid Search CV Tuning for Pearson Correlation

# PearsonCorrelation

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
)
from xgboost import XGBClassifier
import warnings

warnings.filterwarnings("ignore")

# Load datasets
df_train = pd.read_csv("ML_DEG_dataset.csv")
columns_of_interest = ['MT1M', 'ITM2A', 'VSNL1', 'BRCA2', 'AFF3', 'LGALS2', 'PCSK2', 'TMEM100', 'CDH3', 'LIFR', 'FAM107A', 'STMN2', 'ABCA8', 'CENPA', 'ADH1B','target']
df_train = df_train[columns_of_interest]
#df_train = df_train.iloc[:, 1:]
df_train = df_train[[col for col in df_train.columns if col != 'target'] + ['target']]
X_train = df_train.iloc[:, 0:-1]
y_train = df_train.iloc[:, -1]

df_test = pd.read_csv("GSE106582_gene_expression_renamed.csv")
df_test = df_test.iloc[:, 1:]
df_test = df_test[[col for col in df_test.columns if col != 'target'] + ['target']]
X_test = df_test.iloc[:, 0:-1]
y_test = df_test.iloc[:, -1]

# Ensure common features in both train and test sets
common_cols = set(X_train.columns).intersection(X_test.columns)
X_train_common = X_train[list(common_cols)]
X_test_common = X_test[list(common_cols)]

# Hyperparameter grid for each model
param_grids = {
    'Logistic Regression': {
        'C': [0.01, 0.1, 1, 10, 100],
        'penalty': ['l1', 'l2', 'elasticnet'],
        'solver': ['saga'],
        'max_iter': [500]
    },
    'K-Nearest Neighbors': {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    },
    'Random Forest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'Gradient Boosting': {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'min_samples_split': [2, 5]
    },
    'AdaBoost': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 1]
    },
    'Bagging': {
        'n_estimators': [50, 100, 200],
        'max_samples': [0.5, 0.75, 1.0],
        'max_features': [0.5, 0.75, 1.0],
        'bootstrap': [True, False]
    },
    'Extra Trees': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'XGBoost': {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
}

# Define models
models = {
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')

}

# Dictionary to store best parameters
best_params = {}

# Tune and evaluate models
for name, model in models.items():
    print(f"Tuning {name}...")

    # Perform GridSearchCV
    grid_search = GridSearchCV(model, param_grids[name], cv=10, n_jobs=-1, scoring='accuracy')
    grid_search.fit(X_train_common, y_train)

    # Store best parameters
    best_params[name] = grid_search.best_params_
    print(f"Best parameters for {name}: {grid_search.best_params_}\n")

    # Train model with best parameters
    best_model = grid_search.best_estimator_
    best_model.fit(X_train_common, y_train)

    # Predict on test set
    predictions = best_model.predict(X_test_common)

    # Compute metrics
    test_accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted', zero_division=0)
    recall = recall_score(y_test, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=0)
    mcc = matthews_corrcoef(y_test, predictions)

    print(f"{name} Testing Accuracy: {test_accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"MCC: {mcc:.2f}")

    # Print confusion matrix and classification report
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("Classification Report:")
    print(classification_report(y_test, predictions))
    print("-" * 60)

# Print final best parameters
print("\nFinal Best Parameters for All Models:")
for model, params in best_params.items():
    print(f"{model}: {params}")

Tuning Logistic Regression...
Best parameters for Logistic Regression: {'C': 0.01, 'max_iter': 500, 'penalty': 'l2', 'solver': 'saga'}

Logistic Regression Testing Accuracy: 68.04%
Precision: 0.82
Recall: 0.68
F1-Score: 0.67
MCC: 0.51
Confusion Matrix:
[[55 62]
 [ 0 77]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.47      0.64       117
           1       0.55      1.00      0.71        77

    accuracy                           0.68       194
   macro avg       0.78      0.74      0.68       194
weighted avg       0.82      0.68      0.67       194

------------------------------------------------------------
Tuning K-Nearest Neighbors...
Best parameters for K-Nearest Neighbors: {'metric': 'euclidean', 'n_neighbors': 5, 'weights': 'distance'}

K-Nearest Neighbors Testing Accuracy: 89.69%
Precision: 0.91
Recall: 0.90
F1-Score: 0.90
MCC: 0.80
Confusion Matrix:
[[99 18]
 [ 2 75]]
Classification Report:
              precisi

# Random Search CV

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
)
from xgboost import XGBClassifier

import warnings

warnings.filterwarnings("ignore")

# Load datasets
df_train = pd.read_csv("ML_DEG_dataset.csv")
columns_of_interest = ['MT1M', 'ITM2A', 'VSNL1', 'BRCA2', 'AFF3', 'LGALS2', 'PCSK2', 'TMEM100', 'CDH3', 'LIFR', 'FAM107A', 'STMN2', 'ABCA8', 'CENPA', 'ADH1B','target']
df_train = df_train[columns_of_interest]
#df_train = df_train.iloc[:, 1:]
df_train = df_train[[col for col in df_train.columns if col != 'target'] + ['target']]
X_train = df_train.iloc[:, 0:-1]
y_train = df_train.iloc[:, -1]

df_test = pd.read_csv("GSE106582_gene_expression_renamed.csv")
df_test = df_test.iloc[:, 1:]
df_test = df_test[[col for col in df_test.columns if col != 'target'] + ['target']]
X_test = df_test.iloc[:, 0:-1]
y_test = df_test.iloc[:, -1]

# Ensure common features in both train and test sets
common_cols = set(X_train.columns).intersection(X_test.columns)
X_train_common = X_train[list(common_cols)]
X_test_common = X_test[list(common_cols)]

# Hyperparameter grid for each model
param_grids = {
    'Logistic Regression': {
        'C': np.logspace(-3, 2, 10),
        'penalty': ['l1', 'l2', 'elasticnet'],
        'solver': ['saga'],
        'max_iter': [500, 1000, 5000],
        'l1_ratio': np.linspace(0, 1, 5).tolist()
    },
    'K-Nearest Neighbors': {
        'n_neighbors': range(3, 15),
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    },
    'Random Forest': {
        'n_estimators': range(100, 501, 100),
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'Gradient Boosting': {
        'n_estimators': range(100, 501, 100),
        'learning_rate': np.linspace(0.01, 0.2, 5),
        'max_depth': [3, 5, 7, 10],
        'subsample': [0.6, 0.8, 1.0],
        'min_samples_split': [2, 5, 10]
    },
    'AdaBoost': {
        'n_estimators': range(50, 301, 50),
        'learning_rate': np.linspace(0.01, 1, 5)
    },
    'Bagging': {
        'n_estimators': range(50, 301, 50),
        'max_samples': np.linspace(0.5, 1.0, 5),
        'max_features': np.linspace(0.5, 1.0, 5),
        'bootstrap': [True, False]
    },
    'Extra Trees': {
        'n_estimators': range(100, 501, 100),
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
     'XGBoost': {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
}

# Define models
models = {
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')}

# Dictionary to store best parameters
best_params = {}

# Tune and evaluate models using RandomizedSearchCV
for name, model in models.items():
    print(f"Tuning {name}...")

    # Perform RandomizedSearchCV
    random_search = RandomizedSearchCV(model, param_grids[name],
                                       n_iter=20, cv=10,
                                       n_jobs=-1, scoring='accuracy', random_state=7)

    random_search.fit(X_train_common, y_train)

    # Store best parameters
    best_params[name] = random_search.best_params_
    print(f"Best parameters for {name}: {random_search.best_params_}\n")

    # Train model with best parameters
    best_model = random_search.best_estimator_
    best_model.fit(X_train_common, y_train)

    # Predict on test set
    predictions = best_model.predict(X_test_common)

    # Compute metrics
    test_accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted', zero_division=0)
    recall = recall_score(y_test, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=0)
    mcc = matthews_corrcoef(y_test, predictions)

    print(f"{name} Testing Accuracy: {test_accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"MCC: {mcc:.2f}")

    # Print confusion matrix and classification report
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("Classification Report:")
    print(classification_report(y_test, predictions))
    print("-" * 60)

# Print final best parameters
print("\nFinal Best Parameters for All Models:")
for model, params in best_params.items():
    print(f"{model}: {params}")

Tuning Logistic Regression...
Best parameters for Logistic Regression: {'solver': 'saga', 'penalty': 'l1', 'max_iter': 5000, 'l1_ratio': 0.5, 'C': np.float64(0.1668100537200059)}

Logistic Regression Testing Accuracy: 93.30%
Precision: 0.94
Recall: 0.93
F1-Score: 0.93
MCC: 0.86
Confusion Matrix:
[[115   2]
 [ 11  66]]
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.98      0.95       117
           1       0.97      0.86      0.91        77

    accuracy                           0.93       194
   macro avg       0.94      0.92      0.93       194
weighted avg       0.94      0.93      0.93       194

------------------------------------------------------------
Tuning K-Nearest Neighbors...
Best parameters for K-Nearest Neighbors: {'weights': 'uniform', 'n_neighbors': 4, 'metric': 'euclidean'}

K-Nearest Neighbors Testing Accuracy: 93.81%
Precision: 0.94
Recall: 0.94
F1-Score: 0.94
MCC: 0.88
Confusion Matrix:
[[107  10]
 [  2

# Lasso Regression Tuning

# Grid Search CV

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
)
from xgboost import XGBClassifier
from xgboost import XGBClassifier
import warnings

warnings.filterwarnings("ignore")

# Load datasets
df_train = pd.read_csv("ML_DEG_dataset.csv")
columns_of_interest = ['NR3C2', 'BRCA2', 'AFF3', 'LGALS2', 'TMEM100', 'CDH3', 'ABCG2', 'STMN2', 'ABCA8', 'AZGP1', 'TSPAN7', 'ADH1B','target']
df_train = df_train[columns_of_interest]
#df_train = df_train.iloc[:, 1:]
df_train = df_train[[col for col in df_train.columns if col != 'target'] + ['target']]
X_train = df_train.iloc[:, 0:-1]
y_train = df_train.iloc[:, -1]

df_test = pd.read_csv("GSE106582_gene_expression_renamed.csv")
df_test = df_test.iloc[:, 1:]
df_test = df_test[[col for col in df_test.columns if col != 'target'] + ['target']]
X_test = df_test.iloc[:, 0:-1]
y_test = df_test.iloc[:, -1]

# Ensure common features in both train and test sets
common_cols = set(X_train.columns).intersection(X_test.columns)
X_train_common = X_train[list(common_cols)]
X_test_common = X_test[list(common_cols)]

# Hyperparameter grid for each model
param_grids = {
    'Logistic Regression': {
        'C': [0.01, 0.1, 1, 10, 100],
        'penalty': ['l1', 'l2', 'elasticnet'],
        'solver': ['saga'],
        'max_iter': [500]
    },
    'K-Nearest Neighbors': {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    },
    'Random Forest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'Gradient Boosting': {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'min_samples_split': [2, 5]
    },
    'AdaBoost': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 1]
    },
    'Bagging': {
        'n_estimators': [50, 100, 200],
        'max_samples': [0.5, 0.75, 1.0],
        'max_features': [0.5, 0.75, 1.0],
        'bootstrap': [True, False]
    },
    'Extra Trees': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },

    'XGBoost': {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
}

# Define models
models = {
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

# Dictionary to store best parameters
best_params = {}

# Tune and evaluate models
for name, model in models.items():
    print(f"Tuning {name}...")

    # Perform GridSearchCV
    grid_search = GridSearchCV(model, param_grids[name], cv=10, n_jobs=-1, scoring='accuracy')
    grid_search.fit(X_train_common, y_train)

    # Store best parameters
    best_params[name] = grid_search.best_params_
    print(f"Best parameters for {name}: {grid_search.best_params_}\n")

    # Train model with best parameters
    best_model = grid_search.best_estimator_
    best_model.fit(X_train_common, y_train)

    # Predict on test set
    predictions = best_model.predict(X_test_common)

    # Compute metrics
    test_accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted', zero_division=0)
    recall = recall_score(y_test, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=0)
    mcc = matthews_corrcoef(y_test, predictions)

    print(f"{name} Testing Accuracy: {test_accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"MCC: {mcc:.2f}")

    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("Classification Report:")
    print(classification_report(y_test, predictions))
    print("-" * 60)

print("\nFinal Best Parameters for All Models:")
for model, params in best_params.items():
    print(f"{model}: {params}")

Tuning Logistic Regression...
Best parameters for Logistic Regression: {'C': 0.01, 'max_iter': 500, 'penalty': 'l2', 'solver': 'saga'}

Logistic Regression Testing Accuracy: 95.36%
Precision: 0.95
Recall: 0.95
F1-Score: 0.95
MCC: 0.90
Confusion Matrix:
[[115   2]
 [  7  70]]
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.98      0.96       117
           1       0.97      0.91      0.94        77

    accuracy                           0.95       194
   macro avg       0.96      0.95      0.95       194
weighted avg       0.95      0.95      0.95       194

------------------------------------------------------------
Tuning K-Nearest Neighbors...
Best parameters for K-Nearest Neighbors: {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'uniform'}

K-Nearest Neighbors Testing Accuracy: 87.63%
Precision: 0.90
Recall: 0.88
F1-Score: 0.88
MCC: 0.77
Confusion Matrix:
[[94 23]
 [ 1 76]]
Classification Report:
              prec

# Random Search CV

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
)
from xgboost import XGBClassifier
import warnings

warnings.filterwarnings("ignore")

# Load datasets
df_train = pd.read_csv("ML_DEG_dataset.csv")
columns_of_interest =['NR3C2', 'BRCA2', 'AFF3', 'LGALS2', 'TMEM100', 'CDH3', 'ABCG2', 'STMN2', 'ABCA8', 'AZGP1', 'TSPAN7', 'ADH1B','target']
df_train = df_train[columns_of_interest]
#df_train = df_train.iloc[:, 1:]
df_train = df_train[[col for col in df_train.columns if col != 'target'] + ['target']]
X_train = df_train.iloc[:, 0:-1]
y_train = df_train.iloc[:, -1]

df_test = pd.read_csv("GSE106582_gene_expression_renamed.csv")
df_test = df_test.iloc[:, 1:]
df_test = df_test[[col for col in df_test.columns if col != 'target'] + ['target']]
X_test = df_test.iloc[:, 0:-1]
y_test = df_test.iloc[:, -1]

# Ensure common features in both train and test sets
common_cols = set(X_train.columns).intersection(X_test.columns)
X_train_common = X_train[list(common_cols)]
X_test_common = X_test[list(common_cols)]

# Hyperparameter distributions for each model
param_distributions = {
    'Logistic Regression': {
        'C': np.logspace(-2, 2, 10),
        'penalty': ['l1', 'l2'],
        'solver': ['saga'],
        'max_iter': [500]
    },
    'K-Nearest Neighbors': {
        'n_neighbors': range(3, 10),
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    },
    'Random Forest': {
        'n_estimators': range(100, 301, 100),
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'Gradient Boosting': {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'min_samples_split': [2, 5]
    },
    'AdaBoost': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 1]
    },
    'Bagging': {
        'n_estimators': [50, 100, 200],
        'max_samples': [0.5, 0.75, 1.0],
        'max_features': [0.5, 0.75, 1.0],
        'bootstrap': [True, False]
    },
    'Extra Trees': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },

    'XGBoost': {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
}

# Define models
models = {
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

# Dictionary to store best parameters
best_params = {}

# Tune and evaluate models
for name, model in models.items():
    print(f"Tuning {name}...")

    # Perform RandomizedSearchCV
    random_search = RandomizedSearchCV(model, param_distributions[name], n_iter=20, cv=10, n_jobs=-1, scoring='accuracy')
    random_search.fit(X_train_common, y_train)

    # Store best parameters
    best_params[name] = random_search.best_params_
    print(f"Best parameters for {name}: {random_search.best_params_}\n")

    # Train model with best parameters
    best_model = random_search.best_estimator_
    best_model.fit(X_train_common, y_train)

    # Predict on test set
    predictions = best_model.predict(X_test_common)

    # Compute metrics
    test_accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted', zero_division=0)
    recall = recall_score(y_test, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=0)
    mcc = matthews_corrcoef(y_test, predictions)

    print(f"{name} Testing Accuracy: {test_accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"MCC: {mcc:.2f}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("Classification Report:")
    print(classification_report(y_test, predictions))
    print("-" * 60)

print("\nFinal Best Parameters for All Models:")
for model, params in best_params.items():
    print(f"{model}: {params}")

Tuning Logistic Regression...
Best parameters for Logistic Regression: {'solver': 'saga', 'penalty': 'l2', 'max_iter': 500, 'C': np.float64(0.01)}

Logistic Regression Testing Accuracy: 95.36%
Precision: 0.95
Recall: 0.95
F1-Score: 0.95
MCC: 0.90
Confusion Matrix:
[[115   2]
 [  7  70]]
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.98      0.96       117
           1       0.97      0.91      0.94        77

    accuracy                           0.95       194
   macro avg       0.96      0.95      0.95       194
weighted avg       0.95      0.95      0.95       194

------------------------------------------------------------
Tuning K-Nearest Neighbors...
Best parameters for K-Nearest Neighbors: {'weights': 'uniform', 'n_neighbors': 3, 'metric': 'euclidean'}

K-Nearest Neighbors Testing Accuracy: 87.63%
Precision: 0.90
Recall: 0.88
F1-Score: 0.88
MCC: 0.77
Confusion Matrix:
[[94 23]
 [ 1 76]]
Classification Report:
      

# mRMR Feature Selection Tuning


# Grid Search CV

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
)

from xgboost import XGBClassifier
import warnings

warnings.filterwarnings("ignore")

# Load datasets
df_train = pd.read_csv("ML_DEG_dataset.csv")
columns_of_interest = ['CDH3', 'GNG7', 'GDPD3', 'MMP1', 'ABCA8', 'VSNL1', 'ADH1B', 'AZGP1', 'LGALS2', 'TMEM100', 'DPEP1',
                       'AFF3', 'BRCA2', 'MS4A12', 'CENPA', 'LIFR', 'CEP72', 'ABCG2', 'ABI3BP', 'COL10A1', 'MT1M', 'MAD2L1',
                       'STMN2', 'NFE2L3', 'SLC4A4', 'KRT23', 'PRR7', 'target']
df_train = df_train[columns_of_interest]
#df_train = df_train.iloc[:, 1:]
df_train = df_train[[col for col in df_train.columns if col != 'target'] + ['target']]
X_train = df_train.iloc[:, 0:-1]
y_train = df_train.iloc[:, -1]

df_test = pd.read_csv("GSE106582_gene_expression_renamed.csv")
df_test = df_test.iloc[:, 1:]
df_test = df_test[[col for col in df_test.columns if col != 'target'] + ['target']]
X_test = df_test.iloc[:, 0:-1]
y_test = df_test.iloc[:, -1]

# Ensure common features in both train and test sets
common_cols = set(X_train.columns).intersection(X_test.columns)
X_train_common = X_train[list(common_cols)]
X_test_common = X_test[list(common_cols)]

# Hyperparameter grid for each model
param_grids = {
    'Logistic Regression': {
        'C': [0.01, 0.1, 1, 10, 100],
        'penalty': ['l1', 'l2', 'elasticnet'],
        'solver': ['saga'],
        'max_iter': [500]
    },
    'K-Nearest Neighbors': {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    },
    'Random Forest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'Gradient Boosting': {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'min_samples_split': [2, 5]
    },
    'AdaBoost': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 1]
    },
    'Bagging': {
        'n_estimators': [50, 100, 200],
        'max_samples': [0.5, 0.75, 1.0],
        'max_features': [0.5, 0.75, 1.0],
        'bootstrap': [True, False]
    },
    'Extra Trees': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },

    'XGBoost': {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
}

# Define models
models = {
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

# Dictionary to store best parameters
best_params = {}

# Tune and evaluate models
for name, model in models.items():
    print(f"Tuning {name}...")

    # Perform GridSearchCV
    grid_search = GridSearchCV(model, param_grids[name], cv=10, n_jobs=-1, scoring='accuracy')
    grid_search.fit(X_train_common, y_train)

    # Store best parameters
    best_params[name] = grid_search.best_params_
    print(f"Best parameters for {name}: {grid_search.best_params_}\n")

    # Train model with best parameters
    best_model = grid_search.best_estimator_
    best_model.fit(X_train_common, y_train)

    # Predict on test set
    predictions = best_model.predict(X_test_common)

    # Compute metrics
    test_accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted', zero_division=0)
    recall = recall_score(y_test, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=0)
    mcc = matthews_corrcoef(y_test, predictions)

    print(f"{name} Testing Accuracy: {test_accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"MCC: {mcc:.2f}")

    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("Classification Report:")
    print(classification_report(y_test, predictions))
    print("-" * 60)

print("\nFinal Best Parameters for All Models:")
for model, params in best_params.items():
    print(f"{model}: {params}")

Tuning Logistic Regression...
Best parameters for Logistic Regression: {'C': 1, 'max_iter': 500, 'penalty': 'l2', 'solver': 'saga'}

Logistic Regression Testing Accuracy: 87.63%
Precision: 0.90
Recall: 0.88
F1-Score: 0.88
MCC: 0.77
Confusion Matrix:
[[94 23]
 [ 1 76]]
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.80      0.89       117
           1       0.77      0.99      0.86        77

    accuracy                           0.88       194
   macro avg       0.88      0.90      0.88       194
weighted avg       0.90      0.88      0.88       194

------------------------------------------------------------
Tuning K-Nearest Neighbors...
Best parameters for K-Nearest Neighbors: {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'uniform'}

K-Nearest Neighbors Testing Accuracy: 73.20%
Precision: 0.80
Recall: 0.73
F1-Score: 0.73
MCC: 0.53
Confusion Matrix:
[[71 46]
 [ 6 71]]
Classification Report:
              precision  

# Random Search CV

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
)
from xgboost import XGBClassifier
import warnings

warnings.filterwarnings("ignore")

# Load datasets
df_train = pd.read_csv("ML_DEG_dataset.csv")
columns_of_interest = ['CDH3', 'GNG7', 'GDPD3', 'MMP1', 'ABCA8', 'VSNL1', 'ADH1B', 'AZGP1', 'LGALS2', 'TMEM100', 'DPEP1',
                       'AFF3', 'BRCA2', 'MS4A12', 'CENPA', 'LIFR', 'CEP72', 'ABCG2', 'ABI3BP', 'COL10A1', 'MT1M', 'MAD2L1',
                       'STMN2', 'NFE2L3', 'SLC4A4', 'KRT23', 'PRR7', 'target']
df_train = df_train[columns_of_interest]
#df_train = df_train.iloc[:, 1:]
df_train = df_train[[col for col in df_train.columns if col != 'target'] + ['target']]
X_train = df_train.iloc[:, 0:-1]
y_train = df_train.iloc[:, -1]

df_test = pd.read_csv("GSE106582_gene_expression_renamed.csv")
df_test = df_test.iloc[:, 1:]
df_test = df_test[[col for col in df_test.columns if col != 'target'] + ['target']]
X_test = df_test.iloc[:, 0:-1]
y_test = df_test.iloc[:, -1]

# Ensure common features in both train and test sets
common_cols = set(X_train.columns).intersection(X_test.columns)
X_train_common = X_train[list(common_cols)]
X_test_common = X_test[list(common_cols)]

# Hyperparameter distributions for each model
param_distributions = {
    'Logistic Regression': {
        'C': np.logspace(-2, 2, 10),
        'penalty': ['l1', 'l2'],
        'solver': ['saga'],
        'max_iter': [500]
    },
    'K-Nearest Neighbors': {
        'n_neighbors': range(3, 10),
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    },
    'Random Forest': {
        'n_estimators': range(100, 301, 100),
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'Gradient Boosting': {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'min_samples_split': [2, 5]
    },
    'AdaBoost': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 1]
    },
    'Bagging': {
        'n_estimators': [50, 100, 200],
        'max_samples': [0.5, 0.75, 1.0],
        'max_features': [0.5, 0.75, 1.0],
        'bootstrap': [True, False]
    },
    'Extra Trees': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'CatBoost': {
        'iterations': [100, 200, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'depth': [4, 6, 8],
        'l2_leaf_reg': [3, 5, 7]
    },
    'XGBoost': {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
}

# Define models
models = {
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

# Dictionary to store best parameters
best_params = {}

# Tune and evaluate models
for name, model in models.items():
    print(f"Tuning {name}...")

    # Perform RandomizedSearchCV
    random_search = RandomizedSearchCV(model, param_distributions[name], n_iter=20, cv=10, n_jobs=-1, scoring='accuracy')
    random_search.fit(X_train_common, y_train)

    # Store best parameters
    best_params[name] = random_search.best_params_
    print(f"Best parameters for {name}: {random_search.best_params_}\n")

    # Train model with best parameters
    best_model = random_search.best_estimator_
    best_model.fit(X_train_common, y_train)

    # Predict on test set
    predictions = best_model.predict(X_test_common)

    # Compute metrics
    test_accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted', zero_division=0)
    recall = recall_score(y_test, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=0)
    mcc = matthews_corrcoef(y_test, predictions)

    print(f"{name} Testing Accuracy: {test_accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"MCC: {mcc:.2f}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("Classification Report:")
    print(classification_report(y_test, predictions))
    print("-" * 60)

print("\nFinal Best Parameters for All Models:")
for model, params in best_params.items():
    print(f"{model}: {params}")

Tuning Logistic Regression...
Best parameters for Logistic Regression: {'solver': 'saga', 'penalty': 'l1', 'max_iter': 500, 'C': np.float64(1.6681005372000592)}

Logistic Regression Testing Accuracy: 87.63%
Precision: 0.90
Recall: 0.88
F1-Score: 0.88
MCC: 0.77
Confusion Matrix:
[[94 23]
 [ 1 76]]
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.80      0.89       117
           1       0.77      0.99      0.86        77

    accuracy                           0.88       194
   macro avg       0.88      0.90      0.88       194
weighted avg       0.90      0.88      0.88       194

------------------------------------------------------------
Tuning K-Nearest Neighbors...
Best parameters for K-Nearest Neighbors: {'weights': 'distance', 'n_neighbors': 4, 'metric': 'manhattan'}

K-Nearest Neighbors Testing Accuracy: 79.38%
Precision: 0.84
Recall: 0.79
F1-Score: 0.80
MCC: 0.63
Confusion Matrix:
[[82 35]
 [ 5 72]]
Classification Rep

In [ ]:
# GRid Search CV for 15 Dataset

# Elastic Net Feature Selection Tuning

# Grid Search CV

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
)

from xgboost import XGBClassifier
import warnings

warnings.filterwarnings("ignore")

# Load datasets
df_train = pd.read_csv("ML_DEG_dataset.csv")
columns_of_interest = [ 'NEBL.1', 'CDH3', 'LGI1', 'BRCA2', 'PTGDS', 'ASPA', 'ABCG2', 'SST', 'TNFRSF11B', 'SLC4A4', 'TAF1A', 'SPINK2', 'PTGDR', 'AFF3', 'DHRS9', 'TG', 'MMP12', 'ADH1B', 'IL23A', 'ACSL6', 'TSPAN7', 'MMP1', 'PDE9A', 'SPINK5', 'CDKN2A','target']
df_train = df_train[columns_of_interest]
#df_train = df_train.iloc[:, 1:]
df_train = df_train[[col for col in df_train.columns if col != 'target'] + ['target']]
X_train = df_train.iloc[:, 0:-1]
y_train = df_train.iloc[:, -1]

df_test = pd.read_csv("GSE106582_gene_expression_renamed.csv")
df_test = df_test.iloc[:, 1:]
df_test = df_test[[col for col in df_test.columns if col != 'target'] + ['target']]
X_test = df_test.iloc[:, 0:-1]
y_test = df_test.iloc[:, -1]

# Ensure common features in both train and test sets
common_cols = set(X_train.columns).intersection(X_test.columns)
X_train_common = X_train[list(common_cols)]
X_test_common = X_test[list(common_cols)]

# Hyperparameter grid for each model
param_grids = {
    'Logistic Regression': {
        'C': [0.01, 0.1, 1, 10, 100],
        'penalty': ['l1', 'l2', 'elasticnet'],
        'solver': ['saga'],
        'max_iter': [500]
    },
    'K-Nearest Neighbors': {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    },
    'Random Forest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'Gradient Boosting': {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'min_samples_split': [2, 5]
    },
    'AdaBoost': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 1]
    },
    'Bagging': {
        'n_estimators': [50, 100, 200],
        'max_samples': [0.5, 0.75, 1.0],
        'max_features': [0.5, 0.75, 1.0],
        'bootstrap': [True, False]
    },
    'Extra Trees': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },

    'XGBoost': {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
}

# Define models
models = {
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

# Dictionary to store best parameters
best_params = {}

# Tune and evaluate models
for name, model in models.items():
    print(f"Tuning {name}...")

    # Perform GridSearchCV
    grid_search = GridSearchCV(model, param_grids[name], cv=10, n_jobs=-1, scoring='accuracy')
    grid_search.fit(X_train_common, y_train)

    # Store best parameters
    best_params[name] = grid_search.best_params_
    print(f"Best parameters for {name}: {grid_search.best_params_}\n")

    # Train model with best parameters
    best_model = grid_search.best_estimator_
    best_model.fit(X_train_common, y_train)

    # Predict on test set
    predictions = best_model.predict(X_test_common)

    # Compute metrics
    test_accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted', zero_division=0)
    recall = recall_score(y_test, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=0)
    mcc = matthews_corrcoef(y_test, predictions)

    print(f"{name} Testing Accuracy: {test_accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"MCC: {mcc:.2f}")

    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("Classification Report:")
    print(classification_report(y_test, predictions))
    print("-" * 60)

print("\nFinal Best Parameters for All Models:")
for model, params in best_params.items():
    print(f"{model}: {params}")

Tuning Logistic Regression...
Best parameters for Logistic Regression: {'C': 0.01, 'max_iter': 500, 'penalty': 'l1', 'solver': 'saga'}

Logistic Regression Testing Accuracy: 94.33%
Precision: 0.94
Recall: 0.94
F1-Score: 0.94
MCC: 0.88
Confusion Matrix:
[[114   3]
 [  8  69]]
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.97      0.95       117
           1       0.96      0.90      0.93        77

    accuracy                           0.94       194
   macro avg       0.95      0.94      0.94       194
weighted avg       0.94      0.94      0.94       194

------------------------------------------------------------
Tuning K-Nearest Neighbors...
Best parameters for K-Nearest Neighbors: {'metric': 'euclidean', 'n_neighbors': 5, 'weights': 'uniform'}

K-Nearest Neighbors Testing Accuracy: 93.81%
Precision: 0.94
Recall: 0.94
F1-Score: 0.94
MCC: 0.87
Confusion Matrix:
[[109   8]
 [  4  73]]
Classification Report:
              

# Random Search CV

In [ ]:
!pip install xgboost

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier



from xgboost import XGBClassifier
import warnings

warnings.filterwarnings("ignore")

# Load datasets
df_train = pd.read_csv("ML_DEG_dataset.csv")
columns_of_interest = [ 'NEBL.1', 'CDH3', 'LGI1', 'BRCA2', 'PTGDS', 'ASPA', 'ABCG2', 'SST', 'TNFRSF11B', 'SLC4A4', 'TAF1A', 'SPINK2', 'PTGDR', 'AFF3', 'DHRS9', 'TG', 'MMP12', 'ADH1B', 'IL23A', 'ACSL6', 'TSPAN7', 'MMP1', 'PDE9A', 'SPINK5', 'CDKN2A','target']
df_train = df_train[columns_of_interest]
#df_train = df_train.iloc[:, 1:]
df_train = df_train[[col for col in df_train.columns if col != 'target'] + ['target']]
X_train = df_train.iloc[:, 0:-1]
y_train = df_train.iloc[:, -1]

df_test = pd.read_csv("GSE106582_gene_expression_renamed.csv")
df_test = df_test.iloc[:, 1:]
df_test = df_test[[col for col in df_test.columns if col != 'target'] + ['target']]
X_test = df_test.iloc[:, 0:-1]
y_test = df_test.iloc[:, -1]

# Ensure common features in both train and test sets
common_cols = set(X_train.columns).intersection(X_test.columns)
X_train_common = X_train[list(common_cols)]
X_test_common = X_test[list(common_cols)]

# Hyperparameter distributions for each model
param_distributions = {
    'Logistic Regression': {
        'C': np.logspace(-2, 2, 10),
        'penalty': ['l1', 'l2'],
        'solver': ['saga'],
        'max_iter': [500]
    },
    'K-Nearest Neighbors': {
        'n_neighbors': range(3, 10),
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    },
    'Random Forest': {
        'n_estimators': range(100, 301, 100),
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'Gradient Boosting': {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'min_samples_split': [2, 5]
    },
    'AdaBoost': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 1]
    },
    'Bagging': {
        'n_estimators': [50, 100, 200],
        'max_samples': [0.5, 0.75, 1.0],
        'max_features': [0.5, 0.75, 1.0],
        'bootstrap': [True, False]
    },
    'Extra Trees': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },

    'XGBoost': {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
}

# Define models
models = {
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

# Dictionary to store best parameters
best_params = {}

# Tune and evaluate models
for name, model in models.items():
    print(f"Tuning {name}...")

    # Perform RandomizedSearchCV
    random_search = RandomizedSearchCV(model, param_distributions[name], n_iter=20, cv=10, n_jobs=-1, scoring='accuracy')
    random_search.fit(X_train_common, y_train)

    # Store best parameters
    best_params[name] = random_search.best_params_
    print(f"Best parameters for {name}: {random_search.best_params_}\n")

    # Train model with best parameters
    best_model = random_search.best_estimator_
    best_model.fit(X_train_common, y_train)

    # Predict on test set
    predictions = best_model.predict(X_test_common)

    # Compute metrics
    test_accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted', zero_division=0)
    recall = recall_score(y_test, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=0)
    mcc = matthews_corrcoef(y_test, predictions)

    print(f"{name} Testing Accuracy: {test_accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"MCC: {mcc:.2f}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("Classification Report:")
    print(classification_report(y_test, predictions))
    print("-" * 60)

print("\nFinal Best Parameters for All Models:")
for model, params in best_params.items():
    print(f"{model}: {params}")

Tuning Logistic Regression...
Best parameters for Logistic Regression: {'solver': 'saga', 'penalty': 'l1', 'max_iter': 500, 'C': np.float64(0.01)}

Logistic Regression Testing Accuracy: 94.33%
Precision: 0.94
Recall: 0.94
F1-Score: 0.94
MCC: 0.88
Confusion Matrix:
[[114   3]
 [  8  69]]
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.97      0.95       117
           1       0.96      0.90      0.93        77

    accuracy                           0.94       194
   macro avg       0.95      0.94      0.94       194
weighted avg       0.94      0.94      0.94       194

------------------------------------------------------------
Tuning K-Nearest Neighbors...
Best parameters for K-Nearest Neighbors: {'weights': 'uniform', 'n_neighbors': 5, 'metric': 'euclidean'}

K-Nearest Neighbors Testing Accuracy: 93.81%
Precision: 0.94
Recall: 0.94
F1-Score: 0.94
MCC: 0.87
Confusion Matrix:
[[109   8]
 [  4  73]]
Classification Report:
  